In [1]:
from business_models import hahn
import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations, product

In [2]:
query = '''
    USE hahn;
    PRAGMA AnsiInForEmptyOrNullableItemsCollections;

    $fct_delivery = '//home/delivery-dwh/cdm/delivery/fct_delivery';

    $date_start = '2023-06-01';

    SELECT
    initiator_phone_pd_id,
    claim_uuid,
    max(utc_claim_finish_dttm) as utc_claim_finish_dttm
    from RANGE($fct_delivery, $date_start)
    where 1=1
        and utc_order_created_dttm >= "2023-06-01"
        and source_country_name_en = 'Russia'
        
           -- and source_city_name_en = 'Ufa'
        
        and (ListHas(Yson::ConvertToStringList(claim_created_tariff_class_code_list), 'cargo')
            or ListHas(Yson::ConvertToStringList(claim_created_tariff_class_code_list), 'cargocorp')
            or ListHas(Yson::ConvertToStringList(claim_created_tariff_class_code_list), 'cargo_long')
            or ListHas(Yson::ConvertToStringList(claim_created_tariff_class_code_list), 'cargo_express')
            )

        and initiator_phone_pd_id is not null

        and claim_app_name = 'Go'

        and final_claim_status = 'cancelled_by_client'
        and utc_claim_performer_found_dttm is not null
    group by initiator_phone_pd_id, claim_uuid
    ;
'''

In [ ]:
df = hahn(query)

In [33]:
df

,initiator_phone_pd_id,claim_uuid,utc_claim_finish_dttm,next_claim_ts,ts_delta,time_bucket,prev_delta,session_cancel,cancel_number,start_session,time_from_start,prev_time_from_start
0,0000021e2ec246e4abc11b69415ae7fd,91c86da431464aec80726d8acb56f38b,2023-06-27 07:23:52,2023-10-04 07:38:10,142574,NaN,99999999,1,1,2023-06-27 07:23:52,0,0
1,0000021e2ec246e4abc11b69415ae7fd,118724e0c69d48779da794bb0e1dbd51,2023-10-04 07:38:10,2023-11-09 09:57:21,51979,NaN,142574,2,1,2023-10-04 07:38:10,0,0
2,0000021e2ec246e4abc11b69415ae7fd,3c37ff20f8b64e37be1c29fed5a4ff7b,2023-11-09 09:57:21,2023-11-09 12:12:07,134,NaN,51979,3,1,2023-11-09 09:57:21,0,0
3,0000021e2ec246e4abc11b69415ae7fd,587f82bab6f04bf88d89555c0f9af5b1,2023-11-09 12:12:07,9999-01-01 01:01:01,4194520608,NaN,134,4,1,2023-11-09 12:12:07,0,0
4,00001518f4224fa49a016ded16d98bd5,af90fb4cb086404597b2a92e56738c5c,2023-06-13 08:53:01,2023-06-13 08:54:26,1,15.0,99999999,5,1,2023-06-13 08:53:01,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
768424,fffef2f7669b4a70ba87ad7084f1cbaa,8fdfdc950d264c698c64e7808ce33d5b,2023-10-19 05:06:51,9999-01-01 01:01:01,4194551274,NaN,99999999,559794,1,2023-10-19 05:06:51,0,0
768425,ffff414f02fb43a2ba84871abee4b014,be259e9031614bf4b301054ce1ff930c,2023-08-14 06:07:37,2023-08-14 06:11:37,4,15.0,99999999,559795,1,2023-08-14 06:07:37,0,0
768426,ffff414f02fb43a2ba84871abee4b014,cbfb955bc8d74e94bf2e24be5b67ca05,2023-08-14 06:11:37,9999-01-01 01:01:01,4194646249,NaN,4,559795,2,2023-08-14 06:07:37,4,0
768427,ffff573dda4b49b38a261771e59e84ff,fc2776e70f1e47e48ed11cf83a372fd5,2023-10-03 06:26:56,2023-10-03 06:51:45,24,30.0,99999999,559796,1,2023-10-03 06:26:56,0,0


In [34]:
df = df.sort_values(by=['initiator_phone_pd_id','utc_claim_finish_dttm'])

In [35]:
df['next_claim_ts'] = df.sort_values(by=['utc_claim_finish_dttm'],ascending=True).groupby(
    ['initiator_phone_pd_id'])['utc_claim_finish_dttm'].shift(-1)
df['next_claim_ts'].fillna('9999-01-01 01:01:01', inplace=True)

In [36]:
fmt = '%Y-%m-%d %H:%M:%S'

df['ts_delta'] = df.apply(lambda x: (
    datetime.strptime(x.next_claim_ts, fmt) - datetime.strptime(x.utc_claim_finish_dttm, fmt)) // timedelta(minutes=1), axis=1)

In [37]:
def bucket_classifier(x):
    if x <= 15:
        return 15
    elif x <= 30:
        return 30
    elif x <= 45:
        return 45
    elif x <= 60:
        return 60

In [38]:
df['time_bucket'] = df['ts_delta'].apply(bucket_classifier)

In [39]:
df['prev_delta'] = df.sort_values(by=['utc_claim_finish_dttm'],ascending=True).groupby(
    ['initiator_phone_pd_id'])['ts_delta'].shift(1)
df['prev_delta'].fillna(99999999, inplace=True)
df['prev_delta'] = df['prev_delta'].astype(int)

In [40]:
def session_maker(x: pd.Series):
    cnt = 0
    out = []
    for i in x:
        if i > 60:
            cnt += 1
        out.append(cnt)
    return out

In [41]:
df['session_cancel'] = session_maker(df.prev_delta)

In [42]:
# df_test = df[df['initiator_phone_pd_id']=='fff4fafe1bbe4b72a57496ccd93ec659']

In [43]:
df_agg = df[df.time_bucket.notna()].groupby(
    ['initiator_phone_pd_id','time_bucket']).agg({'claim_uuid':'count'}).reset_index()

In [44]:
df_agg

,initiator_phone_pd_id,time_bucket,claim_uuid
0,00001518f4224fa49a016ded16d98bd5,15.0,1
1,0000b3d69f4145d78d9b1e2109172159,15.0,1
2,00012cda5aec4bd488de27e5ca20c1d7,15.0,3
3,000422cebbf443308c1c3f1fe20756de,15.0,1
4,000450ec298e45d393e0f315fed34db9,15.0,2
...,...,...,...
93823,fffbc56ce82e4a6bb7be7b35e1c1be31,15.0,1
93824,fffce44bf0f440dc965a973e518c2057,15.0,1
93825,fffe61e76cf84ea38ed7c333db2064af,15.0,4
93826,ffff414f02fb43a2ba84871abee4b014,15.0,1


In [45]:
final = pd.pivot_table(df_agg[df_agg['claim_uuid']<=10], values='initiator_phone_pd_id',
                       index='time_bucket', columns='claim_uuid', aggfunc=pd.Series.nunique,
                       fill_value=0)

In [46]:
final_share = (final/df_agg.initiator_phone_pd_id.nunique()).round(2)

In [47]:
final

claim_uuid,1,2,3,4,5,6,7,8,9,10
time_bucket,,,,,,,,,,
15.0,35242,12019,5756,3348,2253,1581,995,701,521,437
30.0,14303,1767,393,139,56,19,9,11,4,6
45.0,7184,540,94,24,17,3,5,2,1,0
60.0,4015,196,37,16,4,2,1,3,0,0


In [48]:
final_share

claim_uuid,1,2,3,4,5,6,7,8,9,10
time_bucket,,,,,,,,,,
15.0,0.46,0.16,0.07,0.04,0.03,0.02,0.01,0.01,0.01,0.01
30.0,0.19,0.02,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00
45.0,0.09,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
60.0,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [49]:
cancellation_users = df.groupby(
    'initiator_phone_pd_id').claim_uuid.count().reset_index().sort_values(by='claim_uuid', ascending=False)

In [50]:
cancellation_users_top10_perc = cancellation_users.head(
    round(cancellation_users.initiator_phone_pd_id.nunique()*0.1))

In [51]:
df_agg_top10 = df_agg[
    df_agg['initiator_phone_pd_id'].isin(cancellation_users_top10_perc.initiator_phone_pd_id.tolist())]

In [52]:
final_top10 = pd.pivot_table(df_agg_top10[df_agg_top10['claim_uuid']<=10], values='initiator_phone_pd_id',
                       index='time_bucket', columns='claim_uuid', aggfunc=pd.Series.nunique,
                       fill_value=0)

In [53]:
final_share_top10 = (final_top10/df_agg_top10.initiator_phone_pd_id.nunique()).round(2)

In [54]:
final_top10

claim_uuid,1,2,3,4,5,6,7,8,9,10
time_bucket,,,,,,,,,,
15.0,5914,5508,4575,3348,2253,1581,995,701,521,437
30.0,6627,1554,387,139,56,19,9,11,4,6
45.0,3627,506,93,24,17,3,5,2,1,0
60.0,2216,181,37,16,4,2,1,3,0,0


In [55]:
final_share_top10

claim_uuid,1,2,3,4,5,6,7,8,9,10
time_bucket,,,,,,,,,,
15.0,0.20,0.19,0.15,0.11,0.08,0.05,0.03,0.02,0.02,0.01
30.0,0.22,0.05,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00
45.0,0.12,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
60.0,0.07,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [56]:
df['cancel_number'] = df.sort_values(['utc_claim_finish_dttm'], ascending=True)\
             .groupby(['session_cancel'])\
             .cumcount() + 1

In [57]:
session_cancel = df.groupby('session_cancel')['utc_claim_finish_dttm'].min().reset_index()
df = df.merge(session_cancel, how='left', on='session_cancel')
df.rename(columns={
    'utc_claim_finish_dttm_x':'utc_claim_finish_dttm','utc_claim_finish_dttm_y':'start_session'}, inplace=True)

In [58]:
df['time_from_start'] = df.apply(lambda x: (
    datetime.strptime(x.utc_claim_finish_dttm, fmt) - datetime.strptime(x.start_session, fmt)) // timedelta(minutes=1), axis=1)

TypeError: strptime() argument 1 must be str, not Series

In [ ]:
df['prev_time_from_start'] = df.sort_values(by=['utc_claim_finish_dttm'],ascending=True).groupby(
    ['session_cancel'])['time_from_start'].shift(1)
df['prev_time_from_start'].fillna(0, inplace=True)
df['prev_time_from_start'] = df['prev_time_from_start'].astype(int)

In [ ]:
cans, mins  = [i for i in range(2, 6)], [15, 30, 45, 60]

In [ ]:
map_dict = {'initiator_phone_pd_id':'Affected users', 'claim_uuid':'Affected claims'}
totals = dict(df[['initiator_phone_pd_id', 'claim_uuid']].nunique())

for i in product(cans, mins):
    df_iter = df[(df['cancel_number']>i[0])&(df['prev_time_from_start']<=i[1])]
    print(f'for {i[0]} cancellations in {i[1]} min:')
    for k, v in dict(df_iter[['initiator_phone_pd_id', 'claim_uuid']].nunique()).items():
        print(f'{map_dict.get(k)}: {v}, {(round(v/totals.get(k), 3))} of total')
    print('--------------------------------------')